# LangChain 모듈 - Callback
LangChain은 LLM 애플리케이션의 다양한 단계에 연결할 수 있는 콜백 시스템을 제공합니다.    
이는 로깅, 모니터링, 스트리밍 및 기타 작업에 유용합니다.

- Code 출처 : https://python.langchain.com/docs/modules/callbacks/
- 수정사항 : 설명과 프롬프트 내용을 한글로 변경

In [1]:
# import os
# os.environ["OPENAI_API_KEY"] = "<your OpenAI API key if not set as env var>"

- LangChain은 몇 가지 내장 핸들러를 제공합니다.
- 가장 기본적인 핸들러는 모든 이벤트를 stdout에 기록하는 StdOutCallbackHandler입니다. 

In [2]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

# Constructor callback: First, let's explicitly set the StdOutCallbackHandler when initializing our chain
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.invoke({"number":2})

# Use verbose flag: Then, let's use the `verbose` flag to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.invoke({"number":2})

# Request callbacks: Finally, let's use the request `callbacks` to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({"number":2}, {"callbacks":[handler]})



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


{'number': 2,
 'text': '3\n\n// 1 = 1\n// 2 = 2\n// 3 = 3\n\n// 1 = 1\n// 2 = 2\n// 3 = 3\n// 4 = 4\n\n// 1 = 1\n// 2 = 2\n// 3 = 3\n// 4 = 4\n// 5 = 5\n\n// 1 = 1\n// 2 = 2\n// 3 = 3\n// 4 = 4\n// 5 = 5\n// 6 = 6\n\n// 1 = 1\n// 2 = 2\n// 3 = 3\n// 4 = 4\n// 5 = 5\n// 6 = 6\n// 7 = 7\n\n// 1 = 1\n// 2 = 2\n// 3 = 3\n// 4 = 4\n// 5 = 5\n// 6 = 6\n// 7 = 7\n// 8 = 8\n\n// 1 = 1\n// 2 = 2\n// 3 = 3\n// '}

## 1. Async callbacks

In [3]:
import asyncio
from typing import Any, Dict, List

from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler
from langchain.schema import HumanMessage, LLMResult
from langchain_openai import ChatOpenAI


class MyCustomSyncHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"Sync handler being called in a `thread_pool_executor`: token: {token}")


class MyCustomAsyncHandler(AsyncCallbackHandler):
    """Async callback handler that can be used to handle callbacks from langchain."""

    async def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """Run when chain starts running."""
        print("zzzz....")
        await asyncio.sleep(0.3)
        class_name = serialized["name"]
        print("Hi! I just woke up. Your llm is starting")

    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """Run when chain ends running."""
        print("zzzz....")
        await asyncio.sleep(0.3)
        print("Hi! I just woke up. Your llm is ending")


# To enable streaming, we pass in `streaming=True` to the ChatModel constructor
# Additionally, we pass in a list with our custom handler
chat = ChatOpenAI(
    max_tokens=25,
    streaming=True,
    callbacks=[MyCustomSyncHandler(), MyCustomAsyncHandler()],
)

await chat.agenerate([[HumanMessage(content="Tell me a joke")]])

zzzz....


Error in MyCustomAsyncHandler.on_llm_start callback: KeyError('name')


Sync handler being called in a `thread_pool_executor`: token: 
Sync handler being called in a `thread_pool_executor`: token: Why
Sync handler being called in a `thread_pool_executor`: token:  did
Sync handler being called in a `thread_pool_executor`: token:  the
Sync handler being called in a `thread_pool_executor`: token:  scare
Sync handler being called in a `thread_pool_executor`: token: crow
Sync handler being called in a `thread_pool_executor`: token:  win
Sync handler being called in a `thread_pool_executor`: token:  an
Sync handler being called in a `thread_pool_executor`: token:  award
Sync handler being called in a `thread_pool_executor`: token: ?
Sync handler being called in a `thread_pool_executor`: token:  Because
Sync handler being called in a `thread_pool_executor`: token:  he
Sync handler being called in a `thread_pool_executor`: token:  was
Sync handler being called in a `thread_pool_executor`: token:  outstanding
Sync handler being called in a `thread_pool_executor`: t

LLMResult(generations=[[ChatGeneration(text='Why did the scarecrow win an award? Because he was outstanding in his field!', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='Why did the scarecrow win an award? Because he was outstanding in his field!'))]], llm_output={'token_usage': {}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('016dfe3d-7378-40f5-96cb-a932f223fc5f'))])

## 2. Custom callback handlers

In [4]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI


class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"My custom handler, token: {token}")


# To enable streaming, we pass in `streaming=True` to the ChatModel constructor
# Additionally, we pass in a list with our custom handler
chat = ChatOpenAI(max_tokens=25, streaming=True, callbacks=[MyCustomHandler()])

chat([HumanMessage(content="Tell me a joke")])

D:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


My custom handler, token: 
My custom handler, token: Why
My custom handler, token:  couldn
My custom handler, token: 't
My custom handler, token:  the
My custom handler, token:  bicycle
My custom handler, token:  stand
My custom handler, token:  up
My custom handler, token:  by
My custom handler, token:  itself
My custom handler, token: ?


My custom handler, token: Because
My custom handler, token:  it
My custom handler, token:  was
My custom handler, token:  two
My custom handler, token:  tired
My custom handler, token: !
My custom handler, token: 


AIMessage(content="Why couldn't the bicycle stand up by itself?\n\nBecause it was two tired!")

## 3. Logging to file

In [5]:
%pip install loguru ansi2html 

  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Using cached ansi2html-1.9.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached win32_setctime-1.1.0-py3-none-any.whl (3.6 kB)
Using cached loguru-0.7.2-py3-none-any.whl (62 kB)
Using cached ansi2html-1.9.1-py3-none-any.whl (17 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.callbacks import FileCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from loguru import logger

logfile = "output.log"

logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)

llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

# this chain will both print to stdout (because verbose=True) and write to 'output.log'
# if verbose=False, the FileCallbackHandler will still write to 'output.log'
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler], verbose=True)
answer = chain.run(number=2)
logger.info(answer)

D:\강의자료\생성형AI로AI애플리케이션개발하기\py311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 


2024-02-21 17:11:09.142 | INFO     | __main__:<module>:19 - 3

1 + 3 = 4

1 + 4 = 5

2 + 2 = 4

2 + 3 = 5

3 + 3 = 6



> Finished chain.


In [7]:
from ansi2html import Ansi2HTMLConverter
from IPython.display import HTML, display

with open("output.log", "r") as f:
    content = f.read()

conv = Ansi2HTMLConverter()
html = conv.convert(content, full=True)

display(HTML(html))